# Bulk email automation task

In [1]:
# Import modules
import pandas as pd
import mammoth # to convert docx file to html

import smtplib, ssl  #to establish connection between gmail and python

## email.mime subclasses
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
#my login credentials

email_from = "ankita.mhetre@zoomrx.com"
password ="ewubambrkhdfoqch"

In [3]:
# Import csv that contains credentials of cleints
email_list = pd.read_csv("Menarini Credentials - Bulk email (1).csv")

In [4]:
email_list

,First Name,Email ID,Password,Tenant Link,cc
0,Antonia,amefford@stemline.com,zHkmkYRl,https://menarini.ferma.ai/#/login,planners@zoomrx.com
1,Maria,mdecarvalho@stemline.com,E4HZOw8A,https://menarini.ferma.ai/#/login,planners@zoomrx.com
2,Megan,msell@stemline.com,UV0Fmlm2,https://menarini.ferma.ai/#/login,planners@zoomrx.com
3,Mary,mhohman@stemline.com,9A7q2Cl9,https://menarini.ferma.ai/#/login,planners@zoomrx.com


In [5]:
# Get all the Names, Email Addreses, password,tenant link and cc

all_names = email_list['First Name']
all_emails = email_list['Email ID']
all_password=email_list['Password']
all_links=email_list['Tenant Link']
all_cc=email_list['cc']

In [6]:
# Loop through the all emails and send each of them a customized email
for idx in range(len(all_emails)):

    email_message = MIMEMultipart()
    email_message['From'] = email_from
    email_message['To'] = all_emails[idx]
    email_message['Cc'] = all_cc[idx]
    email_message['Subject']='Access to Ferma.ai'
    
    # Get each records name, email,tenant link and password
    name = all_names[idx]
    email = all_emails[idx]
    links=all_links[idx]
    password_=all_password[idx]
    cc=all_cc[idx]
   
    # convert the document into html format
    with open("Menarini - Client Email Format (1).docx", "rb") as docx_file:
        result = mammoth.convert_to_html(docx_file)
        html = result.value
    
    # replace the tags to customize the email body    
    html = html.replace("&lt;first_name&gt;", all_names[idx])
    html = html.replace('&lt;URL&gt;',all_links[idx])
    html = html.replace('&lt;email_id&gt;',all_emails[idx])
    html = html.replace('&lt;password&gt;',all_password[idx])
    

    # Attach the html doc defined earlier, as a MIMEText html content type to the MIME message
    email_message.attach(MIMEText(html, "html"))
    # Convert it as a string
    email_string = email_message.as_string()

    
    # Connect to the Gmail SMTP server and Send Email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(email_from, password)
       
    # try to send the email,if not successful print the exception message 
        try:
            server.sendmail(email_from, [all_emails[idx],all_cc[idx]], email_string)
            print('Email to {} successfully sent!\n\n'.format(all_emails[idx]))
        
        except Exception as e:
            print('Email to {} could not be sent :( because {}\n\n'.format(all_emails[idx], str(e)))



Email to amefford@stemline.com successfully sent!


Email to mdecarvalho@stemline.com successfully sent!


Email to msell@stemline.com successfully sent!


Email to mhohman@stemline.com successfully sent!




In [ ]:
#close the server connection
server.close()